In [1]:
#@title Importaciones
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import kagglehub, pandas as pd, os
import numpy
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#@title Si estas desde colab
path = kagglehub.dataset_download("svanoo/myanimelist-dataset")

#Seleccionando el archivo con los animes
anime_dataset = pd.read_csv(f"{path}/anime.csv", sep = '\t',on_bad_lines='skip')

In [ ]:
#@title Si no estas en colab

anime_dataset = pd.read_csv("./anime.csv", sep = '\t',on_bad_lines='skip')

In [3]:
#@title importacion de archivos y formateo de columnas

path = kagglehub.dataset_download("svanoo/myanimelist-dataset")

#Seleccionando el archivo con los animes
anime_dataset = pd.read_csv(f"{path}/anime.csv", sep = '\t',on_bad_lines='skip')

anime_dataset.columns = anime_dataset.columns.str.lower()
anime_dataset.columns = anime_dataset.columns.str.replace(' ', '_')

100%|██████████| 1.78G/1.78G [00:25<00:00, 74.7MB/s]

Extracting files...


In [4]:
anime_dataset = anime_dataset[anime_dataset['anime_id'] != 44303] # anime que cumple con los 200 caracteres en synopsis pero no contiene datos utiles para su estudio
anime_dataset_filt = pd.DataFrame(anime_dataset[ anime_dataset["synopsis"].str.len() >= 200 ])
anime_dataset_filt = anime_dataset_filt.sort_values(by='popularity_rank')
anime_dataset_filt.index = range(0,len(anime_dataset_filt))

In [5]:
#@title Eliminando columnas

col_delete_list = ["score_count", "score_rank","type", "status", "num_episodes", "start_date",
                   "end_date", "season", "studios", "clubs", "score_01_count",
                   "score_02_count", "score_03_count", "score_04_count", "score_05_count",
                   "score_06_count", "score_07_count", "score_08_count", "score_09_count",
                   "score_10_count", "num_episodes", "watching_count",
                   "completed_count","on_hold_count","dropped_count"]

filter_columns = list(filter(lambda x: True if x not in col_delete_list else False, list(anime_dataset_filt.columns)))

anime_dataset_filt = anime_dataset_filt[filter_columns]


anime_dataset_filt['longitud_synopsis_caracteres'] = anime_dataset_filt['synopsis'].apply(len)
anime_dataset_filt['genres'] = anime_dataset_filt['genres'].apply(lambda x: x.split('|'))

<ipython-input-5-62a288dbdb3b>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anime_dataset_filt['longitud_synopsis_caracteres'] = anime_dataset_filt['synopsis'].apply(len)
<ipython-input-5-62a288dbdb3b>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anime_dataset_filt['genres'] = anime_dataset_filt['genres'].apply(lambda x: x.split('|'))


In [6]:
data_generos = pd.DataFrame(list(set(elemento for sublista in anime_dataset_filt['genres'] for elemento in sublista)), columns=['genres_name'])

anime_genres_relation = anime_dataset_filt.explode('genres')[["anime_id", "genres"]] # expande anime_dataset_filt para dividir las listas en filas individuales

data_generos = anime_genres_relation.groupby('genres').size().reset_index(name='cantidad') # Agrupamos por el género y contamos la cantidad de animes en cada uno
data_generos.columns = ['genres_name', 'cantidad']

anime_genres_relation

,anime_id,genres
0,16498,Action
0,16498,Drama
0,16498,Fantasy
0,16498,Mystery
0,16498,Military
...,...,...
8504,50710,Comedy
8504,50710,Romance
8504,50710,Sci-Fi
8504,50710,Shounen


In [7]:
#@title Analisis de correlacion entre longitud de sinopsis y la puntuacion
correlacion_longSynop_score = anime_dataset_filt[['longitud_synopsis_caracteres', 'score']].corr()


# plt.scatter(anime_dataset_filt['longitud_synopsis_caracteres'], anime_dataset_filt['score'], color='blue')
# plt.title('Relación entre Longitud de la Sinopsis y Rango de Popularidad')
# plt.xlabel('Longitud de la Sinopsis')
# plt.ylabel('La puntuación')
# plt.show()
anime_dataset_filt[['longitud_synopsis_caracteres', 'score']]
anime_dataset_filt.groupby('score').agg({'score':'count'})
correlacion_longSynop_score
# anime_dataset_filt[["genres", "anime_id"]]

,longitud_synopsis_caracteres,score
longitud_synopsis_caracteres,1.000000,0.302462
score,0.302462,1.000000


In [8]:
#@title Modelo de IA, no ejecutar si estas en vsc


# Cargar un pipeline de clasificación de emociones
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

# Función para analizar emociones de un texto dado
def detectar_emociones(texto):
    resultados = classifier(texto)#aja yo estoy aqui
    # Procesar y mostrar los resultados de manera ordenada
    emociones = {resultado['label']: resultado['score'] for resultado in resultados[0]}
    emociones_ordenadas = sorted(emociones.items(), key=lambda item: item[1], reverse=True)

    return emociones_ordenadas

# Ejemplo de uso
texto = "I'm going under and this time I fear there's no one to save me"

emociones_detectadas = detectar_emociones(texto)

def print_results(emociones_detectadas):
  print("Emociones detectadas:")
  for emocion, puntaje in emociones_detectadas:
      print(f"{emocion}: {puntaje:.2f}")




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [18]:
#@title Otro modelo no ejecutar

#@title segundo modelo

# Cargar el modelo y el tokenizador T5 ajustado para emociones
modelo_id = "mrm8488/t5-base-finetuned-emotion"
tokenizer = AutoTokenizer.from_pretrained(modelo_id)
modelo = AutoModelForSeq2SeqLM.from_pretrained(modelo_id)

# Función para detectar la emoción principal en un texto dado
def detectar_emocion_t5(texto):
    # Preprocesar el texto para el modelo T5
    entradas = tokenizer("emotion: " + texto, return_tensors="pt", padding=True)

    # Generar la emoción usando el modelo
    salida = modelo.generate(**entradas)

    # Decodificar la emoción generada
    emocion = tokenizer.decode(salida[0], skip_special_tokens=True)
    return emocion



tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [19]:
#@title Primer invento xd

anime_dataset_filt[["title", "synopsis"]].head()
synopsis = anime_dataset_filt["synopsis"].head()


for element in anime_dataset_filt[["title", "synopsis"]].head().itertuples():
  print(element.title)
  print(detectar_emocion_t5(element.synopsis))

Shingeki no Kyojin


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


anger
Death Note
anger
Fullmetal Alchemist: Brotherhood
sadness
One Punch Man
surprise
Sword Art Online
joy
